## Combining DataFrames

Previously, we have seem how to analyze and manipulate data in a single DataFrame. However, you will often find data saved into different files and, therefore, you may need to deal with several different pandas DataFrames.<br>
In this session we will explore different ways of combining DataFrames into a single DataFrame.

Let's start loading the `pandas` library, reading two data sets into DataFrames, and having a quick look at the tabular data: ```surveys.csv``` and ```species.csv```

In [ ]:
import pandas as pd

In [ ]:
surveys_df = pd.read_csv("../data/surveys.csv", keep_default_na=False, na_values=[""])
species_df = pd.read_csv("../data/species.csv", keep_default_na=False, na_values=[""])

In [ ]:
print(surveys_df.info())
surveys_df.head()

In [ ]:
print(species_df.info())
species_df.head()

We now have two DataFrames. The first, `surveys_df`, contains information on individuals of a species recorded in a survey, while the second, `species_df`, contains more detailed information on each species.

## Concatenating DataFrames

The first way we will combine DataFrames is **concatenation**, i.e. simply putting DataFrames one after the other either **verically** or **horizontally**.

Concatenation can be used if the DataFrames are similar, meaning that they either have (mostly) the same rows or columns. We will see examples of this later.

To concatenate two DataFrames you will use the function ```pd.concat()```, specifying as arguments the DataFrames to concatenate and ```axis=0``` or ```axis=1``` for vertical or horizontal concatenation, respectively.

Let us first obtain two small DataFrames from the larger `surveys.csv` dataset.

In [ ]:
# Subsetting DataFrames
surveys_df_sub_first10 = surveys_df.head(10)
surveys_df_sub_last10  = surveys_df.tail(10)

We now have two DataFrames, one with the first ten rows of the original dataset, and another with the last ten rows.

### Vertical concatenation

Let's start with **vertical stacking**. In this case the two DataFrames are simply stacked 'on top of' eachother (remember to specify ```axis=0```).
<div>
<img src="pictures/vertical_stacking.jpeg" width="300"/>
</div>
Vertical stacking can be understood as combining two DataFrames that have different sets of the same type of data. In our example, it may be that one field researcher has registered the first ten entries, and another did the last ten. They both put down the same information (weight, species, and so on) of all different individuals. If we combine them, we have one list of twenty records, rather than two lists of ten. 

In [ ]:
# Stack the DataFrames on top of each other
vertical_stack = pd.concat([surveys_df_sub_first10, surveys_df_sub_last10], axis=0)

In [ ]:
print(vertical_stack.info())
vertical_stack

The resulting DataFrame (```vertical_stack```) consists, as expected, of 20 rows. These are the result of the first and last 10 rows of our original DataFrame ```surveys_df```.

You may have noticed that the last ten rows have very high index, not consecutive with the first ten rows. This is because concatenation preserves the indices of the two original DataFrames. If you want a brand new set of indices for your concateneted DataFrame, simply reset the indices using the method ```.reset_index()```. Notice that this adds a column `index` to your DataFrame, that maintains the original index. If you pass `drop=True` into the function, you will avoid the addition of this column.

In [ ]:
vertical_stack.reset_index()

### Exercise 0

In the given example of vertical concatenation, you concatenated two DataFrames with the same columns. What would happen if the two DataFrames to concatenate have different column number and names?

  1. Create a new DataFrame using the last 10 rows of the species DataFrame (`species_df`);
  2. Concatenate vertically `surveys_df_sub_first_10` and your just created DataFrame;
  3. Print the concatenated DataFrame info on the screen. How may rows does it have? What happened to the columns? Explain why you get this result.

### Horizontal concatenation

It's now time to try **horizontal stacking**. In this case the two DataFrames are simply put one after the other (remember to specify ```axis=1```).
<div>
<img src="pictures/horizontal_stacking.jpeg" width="300"/>
</div>
Horizontal stacking can be understood as combining two DataFrames that have different data on the same 'thing'. In our example, it may be that one field researcher has registered the weight and hindfoot length of an individual, and another wrote down their species and sex. They both registered different information of the same individuals. If we combine them, we have one list with all the information of the individual, rather than two lists with partial information.

We now go back to our DataFrames with 10 survey result each, and concatenate those. In this case, as a result, we would expect a DataFrame with the same number of rows of the original ones (10 row) and twice the number of columns (18 columns).

In [ ]:
# Place the DataFrames side by side
horizontal_stack = pd.concat([surveys_df_sub_first10, surveys_df_sub_last10], axis=1)

In [ ]:
print(horizontal_stack.info())
horizontal_stack

Looking at the result of our horizontal concatenation, we may realise that something went wrong. The total number of rows on the resulting DataFrame is 20, instead of 10.

This happens because horizontal stacking will only put things in the same row that actually belong together. It uses the index to determine this and our two DataFrames have different indices (1-9 and 35539-35548 respectively). If we want to force the DataFrames into the form we had in mind, we need to reset the indices of the second DataFrame so that they will match the ones of the first DataFrame.

In [ ]:
surveys_df_sub_last10 = surveys_df_sub_last10.reset_index(drop=True)

In [ ]:
surveys_df_sub_last10

Now that the index has been reset, we can concatenate the two DataFrames.

In [ ]:
horizontal_stack = pd.concat([surveys_df_sub_first10, surveys_df_sub_last10], axis=1)

In [ ]:
print(horizontal_stack.info())
horizontal_stack

### Exercise 1

In the given example of horizontal concatenation, you first concatenated two DataFrame with different indices, then reset the indices of the second one. Based on the outcome of these two cases, try to answer the following questions:
  1. What happens when you concatenate horizontally two DataFrames with different indexing?
  2. What happens when you concatenate horizontally two DataFrames with the same columns?
  3. What happens when you try to select a column of the `horizontal_stack` DataFrame we just created?
  4. How can you select a specific column, when multiple columns share a name?

## Joining DataFrames

Concatenating DataFrames seems a quite "brutal" operation, you simply merge them one after another either verically or horizontally. What about if you want to merge DataFrames according to the value contained in specific columns? The pandas function ```merge()``` performs an operation that in database language is called *join*, the join operation adds the content of one DataFrame to another. There are different types of joins, but the workflow to perform a join operation is always the same:
<ol>
    <li> You identify a *left* and a *right* DataFrames, the ones you want to join;</li>
    <li> You identify in both your left and righ DataFrame a column to join on;</li>
    <li> You choose the type of join;</li>
    <li> You perform the join running the function <code>pd.merge()</code> with the specified inputs and options.</li>
</ol>

Let's see some join example considering two tiny (few rows) DataFrames, our left DataFrame contains general data of European capitals, while our right DataFrame contains weather measuraments for some Dutch towns.

In [ ]:
left_df = pd.read_csv("../data/EU_capitals_tiny.csv", sep=",", header=0)
right_df = pd.read_csv("../data/Netherlands_town_weather_tiny.csv", sep=",", header=0)

In [ ]:
left_df

In [ ]:
right_df

The column we want to perform the join on is the one containing information about the town. In the left DataFrame this has name *Capital* while in the right one *Town*.

In [ ]:
inner_join = pd.merge(left_df,right_df,left_on='Capital',right_on='Town',how='inner')
inner_join

As you may notice, the resulting DataFrame has only one line, the only row that the columns *Capital* and *Town* have in common (*Amsterdam*). This is because an inner join selects only those row values that are **the same** in the two columns (mathematically, an intersection).<br>
The columns of the two DataFrames are merged, even if they have the same name. In our case, both left and right DataFrames have a column with the same name (*Elevation*). After merging, the two columns are preserved, but with a suffix to distinguish them. If you are not happy with the default suffix, you may specify yours in the list of arguments of the ```pd.merge``` functions.<br>

Let's now look at the other joins:

In [ ]:
left_join = pd.merge(left_df,right_df,left_on='Capital',right_on='Town',how='left')
left_join

In [ ]:
right_join = pd.merge(left_df,right_df,left_on='Capital',right_on='Town',how='right')
right_join

In [ ]:
outer_join = pd.merge(left_df,right_df,left_on='Capital',right_on='Town',how='outer')
outer_join

To resume:
- An inner join selects rows that are in common to both left and right selected columns (intersection);
- A left join selects rows that are in common to both left and right selected columns **AND** all the rows of the left DataFrame;
- A right join selects rows that are in common to both left and right selected columns **AND** all the rows of the right DataFrame; 
- An outer join merges the two DataFrames.

To better understand how join works, it may be useful to look at the diagrams below:
<div>
<img src="pictures/joins.jpeg" width="600"/>
</div>
<ul>
    <li> Do you want to select only <strong>common</strong> information between the two DataFrames? Then you would probably need an inner join; </li>
<li> Do you want to add information to your <strong>left</strong> DataFrame? Then you would probably need a left join; </li>
<li> Do you want to add information to your <strong>right</strong> DataFrame? Then you would probably need a right join; </li>
<li> Do you want to get <strong>all</strong> the information from the two DataFrames? Then you would probably need an outer join. </li>
</ul>